In [2]:
# Author: Kyle Larson
# File Name: OVERTRAINED.ipynb
# License: GPL 3
# Objective: The objective of this document is to demonstrate a simple neural network

import csv
import math
import pandas
import re

#This function converts date and time from 'dd-Mth-yyyy hh:mm' to 'yyyymmddhhmm'
def integer_Date_Time(string_1):
    months = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    string_2 =re.search(r'\d+', string_1).group()
    integer_date_time =int(string_2)*10000
    for i in range(3):
        string_1 = string_1.replace(string_2,'')
        if(re.search(r'\d+', string_1)):
            string_2 = re.search(r'\d+', string_1).group()
        if i==0:
            integer_date_time +=int(string_2)*100000000
        if i==1:
            integer_date_time +=int(string_2)*100
        if i==2:
            integer_date_time +=int(string_2)
            regex = r'(?<=-)([\w\.-]+)- :(\d+)'
            match = re.search(regex, string_1)
            if match: 
                for j in range(len(months)):
                    if(re.match(match.group(1), months[j])):
                        integer_date_time+=(j+1)*1000000
    return integer_date_time

#This function cleans Active Calories, Cycling Distance, Distance and Steps files
#reading their raw data and writing it to smaller tidier files by day
def clean_ACCDDS(data_source, i_loc, o_loc):
    d ={ 'Time':[0] , data_source:[0] }
    df = pandas.DataFrame(d, columns=['Time', data_source])
    
    T ='Time'
    D =data_source
    line_count =0
    start_file_index =0
    Nextdate =0
    date =0
    with open(i_loc+data_source+".csv", mode='r') as csv_file:
    
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count =0

        for row in csv_reader:
            if(line_count>0):
                df = df.append({T:integer_Date_Time(row[1]),D:float(row[2])}, ignore_index=True)
                Nextdate = df.loc[line_count][0] - (df.loc[line_count][0])%10000
                #print(str(integer_Date_Time(row[1]))+", "+row[2])
                if(line_count>1 and Nextdate-date >= 10000):
                    with open(o_loc+data_source+str(Nextdate/10000)+".csv", mode='w') as outf:
                        i =start_file_index
                        outf.write("Time, "+data_source+"\n")
                        while(i<line_count):
                            outf.write(str(df.loc[i][0])+", "+str(df.loc[i][1])+"\n")
                            i +=1
                    start_file_index = line_count
            line_count +=1
            date = Nextdate
    return


#___MAIN____
dataSource = ['Active Calories', 'Cycling Distance', 'Distance', 'Steps']
O_loc =r'C:\Users\admin\anaconda3\01 PROJECTS\02 HEART RATE\By Day\\'
I_loc =r'C:\Users\admin\anaconda3\01 PROJECTS\02 HEART RATE\\'
for i in range(len(dataSource)):
    clean_ACCDDS(dataSource[i], I_loc, O_loc)

In [3]:
t='Time'
h='HRV'

In [11]:
#____Borrowed from FileName:HRV_NN.ipynb this function imports the desired output HRV data
def import_HRV(HR_source, timeStamp_S, timeStamp_E):
    HRV =0
    Running_Sum_HR =0
    HRV_sampling_iter =0
    prev_HR =0
    HR_sampling_interval =10
    dfs_HRV =[pandas.DataFrame()]
    while(timeStamp_S < timeStamp_E):
        with open(HR_source+str(timeStamp_S)+".csv", mode='r') as csv_file:

            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count =0
            df_HR = pandas.DataFrame()
            df_HRV = pandas.DataFrame()
            HRV_sampling_iter=0
            Running_Sum_HR=0
            for row in csv_reader:
                if(line_count >0):
                    df_HR = df_HR.append({t:float(row[0]), 'HR':float(row[1])}, ignore_index=True)
                if(HRV_sampling_iter>1):
                    Running_Sum_HR = Running_Sum_HR + pow( ( 1/(float(row[1]))-1/(prev_HR) ), 2)
                
                    Time_interval = float(row[0]) - df_HR.iloc[line_count-HRV_sampling_iter]['Time']
                    #note: large gaps in data, such as HR taken on different days, need to be removed
                    #hours to minutes
                    Time_interval_hours = (Time_interval-Time_interval%100)*60/100
                    #parse the minutes
                    Time_interval_minutes = Time_interval%100
                
                    HRV =60000*pow((Running_Sum_HR)/HRV_sampling_iter, 0.5)
               
                    if(Time_interval_hours + Time_interval_minutes > HR_sampling_interval):
                        #To account for sometimes irregular measurements a normalizing factor is used
                        n_factor = HR_sampling_interval/(Time_interval_hours + Time_interval_minutes)
                        HRV = HRV*n_factor
                        df_HRV = df_HRV.append({t:float(row[0]),h:HRV}, ignore_index=True )
                        HRV_sampling_iter=0
                        Running_Sum_HR=0
                if(line_count>0): 
                    prev_HR = float(row[1])
                HRV_sampling_iter +=1
                line_count +=1
            
    #df_HR.plot(kind='scatter', x='Time', y='HR')
    #df_HRV.plot(kind='scatter', x='Time', y='HRV')
        dfs_HRV.append(df_HRV)
    #print(df_HRV['HRV'].describe())
        timeStamp_S +=1
    return dfs_HRV

#____Main______
TimeStamp_S =20200711.0
TimeStamp_E =20200718.0
hr_source =r"C:\Users\admin\anaconda3\01 PROJECTS\02 HEART RATE\By Day\Heart_Rate_"
HrV_dF =import_HRV(hr_source, TimeStamp_S, TimeStamp_E) #HrV_dF[n] n>0 now being continuous HRV data

In [49]:
ez = []
for i in range(len(HrV_dF)):
    if( i>0 and i<(len(HrV_dF)-1)):
        ez += [[HrV_dF[i].iloc[2][t], HrV_dF[i].mean()[h]]]
print(ez[3][1])
#print(ez.loc[1][t])
#print(ez[t][1])
#print(ez[h][1])

64.55179808058973


In [62]:
#This function encodes a dataframe of HRV data in 3 binary by time
def encode_HRV(ez):
    SCALE = [40,44,48,52,56,60,64]
 
    for i in range(len(ez)):
        if(ez[i][1]<=SCALE[0]):
            if(i == 0): df_out = pandas.DataFrame({t:ez[i][0],h:[[0,0,0]]})
            else: df_out = df_out.append({t:ez[i][0],h:[0,0,0]},ignore_index=True)
        if(ez[i][1]>SCALE[0] and ez[i][1]<=SCALE[1]):
            if(i == 0): df_out = pandas.DataFrame({t:ez[i][0],h:[[0,0,1]]})
            else: df_out = df_out.append({t:ez[i][0],h:[0,0,1]},ignore_index=True)
        if(ez[i][1]>SCALE[1] and ez[i][1]<=SCALE[2]):
            if(i == 0): df_out = pandas.DataFrame({t:ez[i][0],h:[[0,1,0]]})
            else: df_out = df_out.append({t:ez[i][0],h:[0,1,0]},ignore_index=True)
        if(ez[i][1]>SCALE[2] and ez[i][1]<=SCALE[3]):
            if(i == 0): df_out = pandas.DataFrame({t:ez[i][0],h:[[0,1,1]]})
            else: df_out = df_out.append({t:ez[i][0],h:[0,1,1]},ignore_index=True)
        if(ez[i][1]>SCALE[3] and ez[i][1]<=SCALE[4]):
            if(i == 0): df_out = pandas.DataFrame({t:ez[i][0],h:[[1,0,0]]})
            else: df_out = df_out.append({t:ez[i][0],h:[1,0,0]},ignore_index=True)
        if(ez[i][1]>SCALE[4] and ez[i][1]<=SCALE[5]):
            if(i == 0): df_out = pandas.DataFrame({t:ez[i][0],h:[[1,0,1]]})
            else: df_out = df_out.append({t:ez[i][0],h:[1,0,1]},ignore_index=True)
        if(ez[i][1]>SCALE[5] and ez[i][1]<=SCALE[6]):
            if(i == 0): df_out = pandas.DataFrame({t:ez[i][0],h:[[1,1,0]]})
            else: df_out = df_out.append({t:ez[i][0],h:[1,1,0]},ignore_index=True)
        if(ez[i][1]>SCALE[6]):
            if(i == 0): df_out = pandas.DataFrame({t:ez[i][0],h:[[1,1,1]]})
            else: df_out = df_out.append({t:ez[i][0],h:[1,1,1]},ignore_index=True)
    return df_out

#_______Main____
e=encode_HRV(ez)
print(e)
print(ez)

           Time        HRV
0  2.020071e+11  [0, 1, 0]
1  2.020071e+11  [0, 1, 0]
2  2.020071e+11  [0, 0, 0]
3  2.020071e+11  [1, 1, 1]
4  2.020071e+11  [1, 1, 0]
5  2.020072e+11  [0, 1, 1]
[[202007101302.0, 46.432788566972754], [202007110052.0, 47.80279551414721], [202007120100.0, 35.62239036299937], [202007130637.0, 64.55179808058973], [202007140059.0, 61.671303436511884], [202007150121.0, 48.179099602208886]]


52


In [137]:
day = 5
test1_day1 = eHRVdF[day]
test1_day0 = eHRVdF[day-1]
print(test1_day1.loc[:])

            Time        HRV
0   2.020071e+11  [0, 0, 0]
1   2.020071e+11  [0, 1, 0]
2   2.020071e+11  [0, 0, 1]
3   2.020071e+11  [1, 1, 0]
4   2.020071e+11  [1, 1, 1]
..           ...        ...
69  2.020071e+11  [0, 0, 0]
70  2.020071e+11  [0, 0, 1]
71  2.020071e+11  [1, 1, 1]
72  2.020071e+11  [0, 0, 0]
73  2.020071e+11  [0, 0, 0]

[74 rows x 2 columns]


In [63]:
import numpy as np
expected_output = np.array([[0],[1],[1],[0]])
inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
#print([[0],[1],[1],[0]])
#print(expected_output)
#print([[0,0],[0,1],[1,0],[1,1]])

#Expected_Output =[ [test1_day1.loc[0]['HRV'][0],test1_day1.loc[0]['HRV'][1],test1_day1.loc[0]['HRV'][2]] ]
#for i in range(len(test1_day1)):
    #if i>0:
        #Expected_Output.append([test1_day1.loc[i]['HRV'][0],test1_day1.loc[i]['HRV'][1],test1_day1.loc[i]['HRV'][2]])
#EXPECTED_OUTPUT=np.array(Expected_Output)
#print(EXPECTED_OUTPUT)

In [141]:
from pathlib import Path
time_Search = test1_day0.loc[1]['Time']
time_Search_stamp = (time_Search-time_Search%10000)/10000
readFile_location =r"C:\Users\admin\anaconda3\01 Projects\02 HEART RATE\By Day\\"
for j in range(len(dataSource)):
    file = Path(readFile_location+dataSource[j]+str(time_Search_stamp)+".csv")
    if file.is_file():
        with open(readFile_location+dataSource[j]+str(time_Search_stamp)+".csv", mode='r') as csv_file:
    
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count =0

            for row in csv_reader:
                #if(line_count>0):
                line_count +=1
            print(dataSource[j]+str(line_count))

Active Calories6
Distance21
Steps22


202007120019.0


In [3]:
import numpy as np 
#np.random.seed(0)

def sigmoid (x):
    return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

#Input datasets
inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
expected_output = np.array([[0],[1],[1],[0]])

epochs = 10000
lr = 0.1
inputLayerNeurons, hiddenLayerNeurons, outputLayerNeurons = 4,4,3 #originally 2,2,1

#Random weights and bias initialization
hidden_weights = np.random.uniform(size=(inputLayerNeurons,hiddenLayerNeurons))
hidden_bias =np.random.uniform(size=(1,hiddenLayerNeurons))
output_weights = np.random.uniform(size=(hiddenLayerNeurons,outputLayerNeurons))
output_bias = np.random.uniform(size=(1,outputLayerNeurons))

print("Initial hidden weights: ",end='')
print(*hidden_weights)
print("Initial hidden biases: ",end='')
print(*hidden_bias)
print("Initial output weights: ",end='')
print(*output_weights)
print("Initial output biases: ",end='')
print(*output_bias)


#Training algorithm
for _ in range(epochs):
    #Forward Propagation
    hidden_layer_activation = np.dot(inputs,hidden_weights)
    hidden_layer_activation += hidden_bias
    hidden_layer_output = sigmoid(hidden_layer_activation)

    output_layer_activation = np.dot(hidden_layer_output,output_weights)
    output_layer_activation += output_bias
    predicted_output = sigmoid(output_layer_activation)

    #Backpropagation
    error = expected_output - predicted_output
    d_predicted_output = error * sigmoid_derivative(predicted_output)
    
    error_hidden_layer = d_predicted_output.dot(output_weights.T)
    d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)

    #Updating Weights and Biases
    output_weights += hidden_layer_output.T.dot(d_predicted_output) * lr
    output_bias += np.sum(d_predicted_output,axis=0,keepdims=True) * lr
    hidden_weights += inputs.T.dot(d_hidden_layer) * lr
    hidden_bias += np.sum(d_hidden_layer,axis=0,keepdims=True) * lr

print("Final hidden weights: ",end='')
print(*hidden_weights)
print("Final hidden bias: ",end='')
print(*hidden_bias)
print("Final output weights: ",end='')
print(*output_weights)
print("Final output bias: ",end='')
print(*output_bias)

print("\nOutput from neural network after 10,000 epochs: ",end='')
print(*predicted_output)

Initial hidden weights: [0.23958466 0.52927662] [0.58024919 0.44905656]
Initial hidden biases: [0.40877091 0.15095802]
Initial output weights: [0.97842112] [0.78358121]
Initial output biases: [0.51151716]
Final hidden weights: [5.8975741  3.10911501] [5.89639697 3.10900516]
Final hidden bias: [-1.98213287 -4.64255323]
Final output weights: [6.53359328] [-6.55796747]
Final output bias: [-3.07075352]

Output from neural network after 10,000 epochs: [0.08770912] [0.89759414] [0.89759803] [0.12229463]
